In [1]:
import cv2
import numpy as np

from keras.models import Model
from keras.layers import Activation, Dropout, Dense, Input, Flatten

Using TensorFlow backend.


# load data

In [2]:
smoke_dataset_image_1 = np.load('smoke_data/set1_48_data.npy')
smoke_dataset_label_1 = np.load('smoke_data/set1_48_label.npy')
smoke_dataset_image_2 = np.load('smoke_data/set2_48_data.npy')
smoke_dataset_label_2 = np.load('smoke_data/set2_48_label.npy')
smoke_dataset_image_3 = np.load('smoke_data/set3_48_data.npy')
smoke_dataset_label_3 = np.load('smoke_data/set3_48_label.npy')
smoke_dataset_image_4 = np.load('smoke_data/set4_48_data.npy')
smoke_dataset_label_4 = np.load('smoke_data/set4_48_label.npy')

In [3]:
training_set = smoke_dataset_image_3 * 255
training_set_label = smoke_dataset_label_3.flatten().astype(np.int64)
test_set = smoke_dataset_image_4 * 255
test_set_label = smoke_dataset_label_4.flatten().astype(np.int64)

# HOG

## Get Feature

In [4]:
from skimage import feature as ft

In [5]:
training_set_hog_feature = np.array([ft.hog(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)) for i in training_set])

test_set_hog_feature = np.array([ft.hog(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)) for i in test_set])

/home/song/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


## Construct Fully Connected Network

In [6]:
smoke_detector_hog_input = Input(shape=(training_set_hog_feature.shape[1], ))

In [7]:
smoke_detector_hog_dense = Dense(128, activation='sigmoid')(smoke_detector_hog_input)

In [8]:
smoke_detector_hog_dropout = Dropout(0.3)(smoke_detector_hog_dense)

In [9]:
smoke_detector_hog_prediction = Dense(1, activation='sigmoid')(smoke_detector_hog_dropout)

In [10]:
smoke_detector_hog_model = Model(inputs=smoke_detector_hog_input, outputs=smoke_detector_hog_prediction)

In [11]:
smoke_detector_hog_model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [39]:
smoke_detector_hog_model.fit(training_set_hog_feature, training_set_label, epochs=30)

Epoch 1/30
1383/1383 [==============================] - 0s 155us/step - loss: 0.7190 - acc: 0.5553
Epoch 2/30
1383/1383 [==============================] - 0s 95us/step - loss: 0.6716 - acc: 0.5936
Epoch 3/30
1383/1383 [==============================] - 0s 96us/step - loss: 0.6393 - acc: 0.6016
Epoch 4/30
1383/1383 [==============================] - 0s 91us/step - loss: 0.6094 - acc: 0.6544
Epoch 5/30
1383/1383 [==============================] - 0s 91us/step - loss: 0.5822 - acc: 0.6797
Epoch 6/30
1383/1383 [==============================] - 0s 94us/step - loss: 0.5822 - acc: 0.6544
Epoch 7/30
1383/1383 [==============================] - 0s 93us/step - loss: 0.5764 - acc: 0.6558
Epoch 8/30
1383/1383 [==============================] - 0s 102us/step - loss: 0.5521 - acc: 0.7028
Epoch 9/30
1383/1383 [==============================] - 0s 97us/step - loss: 0.5365 - acc: 0.6992
Epoch 10/30
1383/1383 [==============================] - 0s 91us/step - loss: 0.5272 - acc: 0.7122
Epoch 11/30
1383/

## Test HOG Model

In [40]:
smoke_detector_hog_model.evaluate(training_set_hog_feature, training_set_label)

1383/1383 [==============================] - 0s 43us/step


[0.41491538544797585, 0.80694143179957623]

In [41]:
training_set_label_prediction_hog = smoke_detector_hog_model.predict(training_set_hog_feature)
test_set_label_prediction_hog = smoke_detector_hog_model.predict(test_set_hog_feature)

In [42]:
training_set_label_prediction_hog_positive = training_set_label_prediction_hog[training_set_label == 1]
training_set_label_prediction_hog_negative = training_set_label_prediction_hog[training_set_label == 0]

training_set_label_prediction_hog_positive_true = training_set_label_prediction_hog_positive[training_set_label_prediction_hog_positive >= 0.5]
training_set_label_prediction_hog_positive_false = training_set_label_prediction_hog_positive[training_set_label_prediction_hog_positive < 0.5]
training_set_label_prediction_hog_negative_true = training_set_label_prediction_hog_negative[training_set_label_prediction_hog_negative < 0.5]
training_set_label_prediction_hog_negative_false = training_set_label_prediction_hog_negative[training_set_label_prediction_hog_negative >= 0.5]

print(
    '''HOG+FC Training Set 混淆矩阵 Training Set
    real/predict   True    False
    
    smoke          {0}     {1}
    no smoke       {2}     {3}
    检测率：{4}
    虚警率：{5}'''.format(
    training_set_label_prediction_hog_positive_true.shape[0],
    training_set_label_prediction_hog_positive_false.shape[0],
    training_set_label_prediction_hog_negative_true.shape[0],
    training_set_label_prediction_hog_negative_false.shape[0],
    training_set_label_prediction_hog_positive_true.shape[0] / training_set_label_prediction_hog_positive.shape[0],
    training_set_label_prediction_hog_negative_false.shape[0] / training_set_label_prediction_hog_negative.shape[0]
    )
)

混淆矩阵 Training Set
    real/predict   True    False
    
    smoke          442     110
    no smoke       674     157
    检测率：0.8007246376811594
    虚警率：0.18892900120336945


In [43]:
test_set_label_prediction_hog_positive = test_set_label_prediction_hog[test_set_label == 1]
test_set_label_prediction_hog_negative = test_set_label_prediction_hog[test_set_label == 0]

test_set_label_prediction_hog_positive_true = test_set_label_prediction_hog_positive[test_set_label_prediction_hog_positive >= 0.5]
test_set_label_prediction_hog_positive_false = test_set_label_prediction_hog_positive[test_set_label_prediction_hog_positive < 0.5]
test_set_label_prediction_hog_negative_true = test_set_label_prediction_hog_negative[test_set_label_prediction_hog_negative < 0.5]
test_set_label_prediction_hog_negative_false = test_set_label_prediction_hog_negative[test_set_label_prediction_hog_negative >= 0.5]

print(
    '''HOG+FC Test Set 混淆矩阵 test Set
    real/predict   True    False
    
    smoke          {0}     {1}
    no smoke       {2}     {3}
    检测率：{4}
    虚警率：{5}'''.format(
    test_set_label_prediction_hog_positive_true.shape[0],
    test_set_label_prediction_hog_positive_false.shape[0],
    test_set_label_prediction_hog_negative_true.shape[0],
    test_set_label_prediction_hog_negative_false.shape[0],
    test_set_label_prediction_hog_positive_true.shape[0] / test_set_label_prediction_hog_positive.shape[0],
    test_set_label_prediction_hog_negative_false.shape[0] / test_set_label_prediction_hog_negative.shape[0]
    )
)

混淆矩阵 test Set
    real/predict   True    False
    
    smoke          504     184
    no smoke       647     170
    检测率：0.7325581395348837
    虚警率：0.20807833537331702


## Extract Feature

In [12]:
smoke_detector_hog_model_feature_extraction = Model(inputs=smoke_detector_hog_input, outputs=smoke_detector_hog_dense)

In [45]:
smoke_detector_hog_model_feature_extraction.predict(training_set_hog_feature).shape

(1383, 128)

# LBP

## Get Feature

In [13]:
class LocalBinaryPatterns:
	def __init__(self, numPoints, radius):
		# store the number of points and radius
		self.numPoints = numPoints
		self.radius = radius
 
	def describe(self, image, eps=1e-7):
		# compute the Local Binary Pattern representation
		# of the image, and then use the LBP representation
		# to build the histogram of patterns
		lbp = ft.local_binary_pattern(image, self.numPoints,
			self.radius, method="uniform")
		(hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, self.numPoints + 3),
			range=(0, self.numPoints + 2))
 
		# normalize the histogram
		hist = hist.astype("float")
		hist /= (hist.sum() + eps)
 
		# return the histogram of Local Binary Patterns
		return hist

In [14]:
lbp_descriptor = LocalBinaryPatterns(16, 2)

In [15]:
training_set_lbp_feature = np.array([lbp_descriptor.describe(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)) for i in training_set])

test_set_lbp_feature = np.array([lbp_descriptor.describe(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)) for i in test_set])

In [16]:
smoke_detector_lbp_input = Input(shape=(training_set_lbp_feature.shape[1], ))

In [17]:
smoke_detector_lbp_dense = Dense(128, activation='sigmoid')(smoke_detector_lbp_input)

In [18]:
smoke_detector_lbp_dropout = Dropout(0.3)(smoke_detector_lbp_dense)

In [19]:
smoke_detector_lbp_prediction = Dense(1, activation='sigmoid')(smoke_detector_lbp_dropout)

In [20]:
smoke_detector_lbp_model = Model(inputs=smoke_detector_lbp_input, outputs=smoke_detector_lbp_prediction)

In [21]:
smoke_detector_lbp_model.compile(optimizer='AdaDelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [55]:
smoke_detector_lbp_model.fit(training_set_lbp_feature, training_set_label, epochs=30)

Epoch 1/30
1383/1383 [==============================] - 0s 99us/step - loss: 0.6906 - acc: 0.5770
Epoch 2/30
1383/1383 [==============================] - 0s 31us/step - loss: 0.6788 - acc: 0.5813
Epoch 3/30
1383/1383 [==============================] - 0s 33us/step - loss: 0.6817 - acc: 0.5879
Epoch 4/30
1383/1383 [==============================] - 0s 31us/step - loss: 0.6647 - acc: 0.5842
Epoch 5/30
1383/1383 [==============================] - 0s 30us/step - loss: 0.6679 - acc: 0.5850
Epoch 6/30
1383/1383 [==============================] - 0s 31us/step - loss: 0.6585 - acc: 0.6146
Epoch 7/30
1383/1383 [==============================] - 0s 30us/step - loss: 0.6573 - acc: 0.6132
Epoch 8/30
1383/1383 [==============================] - 0s 33us/step - loss: 0.6587 - acc: 0.6124
Epoch 9/30
1383/1383 [==============================] - 0s 34us/step - loss: 0.6449 - acc: 0.6471
Epoch 10/30
1383/1383 [==============================] - 0s 31us/step - loss: 0.6421 - acc: 0.6276
Epoch 11/30
1383/13

## Test LBP Model

In [56]:
smoke_detector_lbp_model.evaluate(training_set_lbp_feature, training_set_label)

1383/1383 [==============================] - 0s 37us/step


[0.46117093777708273, 0.86695589302935983]

In [57]:
training_set_label_prediction_lbp = smoke_detector_lbp_model.predict(training_set_lbp_feature)
test_set_label_prediction_lbp = smoke_detector_lbp_model.predict(test_set_lbp_feature)

In [58]:
training_set_label_prediction_lbp_positive = training_set_label_prediction_lbp[training_set_label == 1]
training_set_label_prediction_lbp_negative = training_set_label_prediction_lbp[training_set_label == 0]

training_set_label_prediction_lbp_positive_true = training_set_label_prediction_lbp_positive[training_set_label_prediction_lbp_positive >= 0.5]
training_set_label_prediction_lbp_positive_false = training_set_label_prediction_lbp_positive[training_set_label_prediction_lbp_positive < 0.5]
training_set_label_prediction_lbp_negative_true = training_set_label_prediction_lbp_negative[training_set_label_prediction_lbp_negative < 0.5]
training_set_label_prediction_lbp_negative_false = training_set_label_prediction_lbp_negative[training_set_label_prediction_lbp_negative >= 0.5]

print(
    '''LBP+FC Training Set 混淆矩阵 Training Set
    real/predict   True    False
    
    smoke          {0}     {1}
    no smoke       {2}     {3}
    检测率：{4}
    虚警率：{5}'''.format(
    training_set_label_prediction_lbp_positive_true.shape[0],
    training_set_label_prediction_lbp_positive_false.shape[0],
    training_set_label_prediction_lbp_negative_true.shape[0],
    training_set_label_prediction_lbp_negative_false.shape[0],
    training_set_label_prediction_lbp_positive_true.shape[0] / training_set_label_prediction_lbp_positive.shape[0],
    training_set_label_prediction_lbp_negative_false.shape[0] / training_set_label_prediction_lbp_negative.shape[0]
    )
)

混淆矩阵 Training Set
    real/predict   True    False
    
    smoke          425     127
    no smoke       774     57
    检测率：0.769927536231884
    虚警率：0.06859205776173286


In [59]:
test_set_label_prediction_lbp_positive = test_set_label_prediction_lbp[test_set_label == 1]
test_set_label_prediction_lbp_negative = test_set_label_prediction_lbp[test_set_label == 0]

test_set_label_prediction_lbp_positive_true = test_set_label_prediction_lbp_positive[test_set_label_prediction_lbp_positive >= 0.5]
test_set_label_prediction_lbp_positive_false = test_set_label_prediction_lbp_positive[test_set_label_prediction_lbp_positive < 0.5]
test_set_label_prediction_lbp_negative_true = test_set_label_prediction_lbp_negative[test_set_label_prediction_lbp_negative < 0.5]
test_set_label_prediction_lbp_negative_false = test_set_label_prediction_lbp_negative[test_set_label_prediction_lbp_negative >= 0.5]

print(
    '''LBP+FC Test Set 混淆矩阵 test Set
    real/predict   True    False
    
    smoke          {0}     {1}
    no smoke       {2}     {3}
    检测率：{4}
    虚警率：{5}'''.format(
    test_set_label_prediction_lbp_positive_true.shape[0],
    test_set_label_prediction_lbp_positive_false.shape[0],
    test_set_label_prediction_lbp_negative_true.shape[0],
    test_set_label_prediction_lbp_negative_false.shape[0],
    test_set_label_prediction_lbp_positive_true.shape[0] / test_set_label_prediction_lbp_positive.shape[0],
    test_set_label_prediction_lbp_negative_false.shape[0] / test_set_label_prediction_lbp_negative.shape[0]
    )
)

混淆矩阵 test Set
    real/predict   True    False
    
    smoke          575     113
    no smoke       804     13
    检测率：0.8357558139534884
    虚警率：0.01591187270501836


## Extract Feature

In [22]:
smoke_detector_lbp_model_feature_extraction = Model(inputs=smoke_detector_lbp_input, outputs=smoke_detector_lbp_dense)

In [61]:
smoke_detector_lbp_model_feature_extraction.predict(training_set_lbp_feature).shape

(1383, 128)

# VGG16

## Get Feature

In [4]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [5]:
model_vgg16 = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

In [7]:
training_set_224 = np.array([cv2.resize(i, (224, 224)) for i in smoke_dataset_image_3]) * 255
test_set_224 = np.array([cv2.resize(i, (224, 224)) for i in smoke_dataset_image_4]) * 255

In [10]:
training_set_vgg16_feature = model_vgg16.predict(preprocess_input(training_set_224))
#test_set_vgg16_feature = model_vgg16.predict(preprocess_input(test_set_224))

In [23]:
smoke_detector_vgg16_input = Input(shape=(7, 7, 512,))

In [24]:
smoke_detector_vgg16_flatten = Flatten()(smoke_detector_vgg16_input)

In [25]:
smoke_detector_vgg16_dense_1 = Dense(1024, activation='sigmoid')(smoke_detector_vgg16_flatten)

In [26]:
smoke_detector_vgg16_dropout_1 = Dropout(0.3)(smoke_detector_vgg16_dense_1)

In [27]:
smoke_detector_vgg16_dense_2 = Dense(128, activation='sigmoid')(smoke_detector_vgg16_dropout_1)

In [28]:
smoke_detector_vgg16_dropout_2 = Dropout(0.3)(smoke_detector_vgg16_dense_2)

In [29]:
smoke_detector_vgg16_prediction = Dense(1, activation='sigmoid')(smoke_detector_vgg16_dropout_2)

In [30]:
smoke_detector_vgg16_model = Model(inputs=smoke_detector_vgg16_input, outputs=smoke_detector_vgg16_prediction)

In [31]:
smoke_detector_vgg16_model.compile(optimizer='AdaDelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
smoke_detector_vgg16_model.fit(training_set_vgg16_feature, training_set_label, epochs=10)

Epoch 1/10
1383/1383 [==============================] - 15s 11ms/step - loss: 0.3060 - acc: 0.8670
Epoch 2/10
1383/1383 [==============================] - 15s 11ms/step - loss: 0.1095 - acc: 0.9559
Epoch 3/10
1383/1383 [==============================] - 15s 11ms/step - loss: 0.0685 - acc: 0.9754
Epoch 4/10
1383/1383 [==============================] - 15s 11ms/step - loss: 0.0521 - acc: 0.9834
Epoch 5/10
1383/1383 [==============================] - 15s 11ms/step - loss: 0.0326 - acc: 0.9899
Epoch 6/10
1383/1383 [==============================] - 15s 11ms/step - loss: 0.0245 - acc: 0.9935
Epoch 7/10
1383/1383 [==============================] - 15s 11ms/step - loss: 0.0185 - acc: 0.9913
Epoch 8/10
1383/1383 [==============================] - 15s 11ms/step - loss: 0.0149 - acc: 0.9964
Epoch 9/10
1383/1383 [==============================] - 15s 11ms/step - loss: 0.0127 - acc: 0.9964
Epoch 10/10
1383/1383 [==============================] - 15s 11ms/step - loss: 0.0085 - acc: 0.9986


## Test VGG16 Model

In [23]:
test_set_vgg16_feature = model_vgg16.predict(preprocess_input(test_set_224))

In [24]:
training_set_label_prediction_vgg16 = smoke_detector_vgg16_model.predict(training_set_vgg16_feature)
test_set_label_prediction_vgg16 = smoke_detector_vgg16_model.predict(test_set_vgg16_feature)

In [25]:
training_set_label_prediction_vgg16_positive = training_set_label_prediction_vgg16[training_set_label == 1]
training_set_label_prediction_vgg16_negative = training_set_label_prediction_vgg16[training_set_label == 0]

training_set_label_prediction_vgg16_positive_true = training_set_label_prediction_vgg16_positive[training_set_label_prediction_vgg16_positive >= 0.5]
training_set_label_prediction_vgg16_positive_false = training_set_label_prediction_vgg16_positive[training_set_label_prediction_vgg16_positive < 0.5]
training_set_label_prediction_vgg16_negative_true = training_set_label_prediction_vgg16_negative[training_set_label_prediction_vgg16_negative < 0.5]
training_set_label_prediction_vgg16_negative_false = training_set_label_prediction_vgg16_negative[training_set_label_prediction_vgg16_negative >= 0.5]

print(
    '''VGG16+FC Training Set 混淆矩阵 Training Set
    real/predict   True    False
    
    smoke          {0}     {1}
    no smoke       {2}     {3}
    检测率：{4}
    虚警率：{5}'''.format(
    training_set_label_prediction_vgg16_positive_true.shape[0],
    training_set_label_prediction_vgg16_positive_false.shape[0],
    training_set_label_prediction_vgg16_negative_true.shape[0],
    training_set_label_prediction_vgg16_negative_false.shape[0],
    training_set_label_prediction_vgg16_positive_true.shape[0] / training_set_label_prediction_vgg16_positive.shape[0],
    training_set_label_prediction_vgg16_negative_false.shape[0] / training_set_label_prediction_vgg16_negative.shape[0]
    )
)

混淆矩阵 Training Set
    real/predict   True    False
    
    smoke          552     0
    no smoke       831     0
    检测率：1.0
    虚警率：0.0


In [26]:
test_set_label_prediction_vgg16_positive = test_set_label_prediction_vgg16[test_set_label == 1]
test_set_label_prediction_vgg16_negative = test_set_label_prediction_vgg16[test_set_label == 0]

test_set_label_prediction_vgg16_positive_true = test_set_label_prediction_vgg16_positive[test_set_label_prediction_vgg16_positive >= 0.5]
test_set_label_prediction_vgg16_positive_false = test_set_label_prediction_vgg16_positive[test_set_label_prediction_vgg16_positive < 0.5]
test_set_label_prediction_vgg16_negative_true = test_set_label_prediction_vgg16_negative[test_set_label_prediction_vgg16_negative < 0.5]
test_set_label_prediction_vgg16_negative_false = test_set_label_prediction_vgg16_negative[test_set_label_prediction_vgg16_negative >= 0.5]

print(
    '''VGG16+FC Test Set 混淆矩阵 test Set
    real/predict   True    False
    
    smoke          {0}     {1}
    no smoke       {2}     {3}
    检测率：{4}
    虚警率：{5}'''.format(
    test_set_label_prediction_vgg16_positive_true.shape[0],
    test_set_label_prediction_vgg16_positive_false.shape[0],
    test_set_label_prediction_vgg16_negative_true.shape[0],
    test_set_label_prediction_vgg16_negative_false.shape[0],
    test_set_label_prediction_vgg16_positive_true.shape[0] / test_set_label_prediction_vgg16_positive.shape[0],
    test_set_label_prediction_vgg16_negative_false.shape[0] / test_set_label_prediction_vgg16_negative.shape[0]
    )
)

混淆矩阵 test Set
    real/predict   True    False
    
    smoke          685     3
    no smoke       786     31
    检测率：0.9956395348837209
    虚警率：0.037943696450428395


## Extract Feature

In [32]:
smoke_detector_vgg16_model_feature_extraction = Model(inputs=smoke_detector_vgg16_input, outputs=smoke_detector_vgg16_dense_2)

In [29]:
smoke_detector_vgg16_model_feature_extraction.predict(training_set_vgg16_feature).shape

(1383, 128)

# Feature Fusion

In [33]:
from keras.layers import concatenate

In [34]:
smoke_detector_feature_fusion_concat = concatenate([smoke_detector_hog_model_feature_extraction.output, smoke_detector_lbp_model_feature_extraction.output, smoke_detector_vgg16_model_feature_extraction.output])

In [35]:
smoke_detector_feature_fusion_dense_1 = Dense(16, activation='sigmoid')(smoke_detector_feature_fusion_concat)

In [36]:
smoke_detector_feature_fusion_dropout_1 = Dropout(0.2)(smoke_detector_feature_fusion_dense_1)

In [37]:
smoke_detector_feature_fusion_prediction = Dense(1, activation='sigmoid')(smoke_detector_feature_fusion_dropout_1)

In [38]:
smoke_detector_feature_fusion_model = Model(inputs=[smoke_detector_hog_input, smoke_detector_lbp_input, smoke_detector_vgg16_input], outputs=[smoke_detector_feature_fusion_prediction])

In [39]:
smoke_detector_feature_fusion_model.compile(optimizer='AdaDelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [92]:
smoke_detector_feature_fusion_model.fit([training_set_hog_feature, training_set_lbp_feature, training_set_vgg16_feature], training_set_label, epochs=10)

Epoch 1/5
1383/1383 [==============================] - 15s 11ms/step - loss: 0.0130 - acc: 1.0000
Epoch 2/5
1383/1383 [==============================] - 15s 11ms/step - loss: 0.0113 - acc: 1.0000
Epoch 3/5
1383/1383 [==============================] - 15s 11ms/step - loss: 0.0087 - acc: 1.0000
Epoch 4/5
1383/1383 [==============================] - 15s 11ms/step - loss: 0.0081 - acc: 1.0000
Epoch 5/5
1383/1383 [==============================] - 15s 11ms/step - loss: 0.0068 - acc: 1.0000


In [40]:
from keras.utils.vis_utils import plot_model

In [41]:
plot_model(smoke_detector_feature_fusion_model, to_file='model.png', show_shapes=True)

In [86]:
smoke_detector_hog_model.save('model/smoke_detector_hog_model.h5')
smoke_detector_lbp_model.save('model/smoke_detector_lbp_model.h5')
smoke_detector_vgg16_model.save('model/smoke_detector_vgg16_model.h5')
smoke_detector_feature_fusion_model.save('model/smoke_detector_feature_fusion_model.h5')

## Test Feature Fusion Model

In [93]:
training_set_label_prediction_feature_fusion = smoke_detector_feature_fusion_model.predict([training_set_hog_feature, training_set_lbp_feature, training_set_vgg16_feature])
test_set_label_prediction_feature_fusion = smoke_detector_feature_fusion_model.predict([test_set_hog_feature, test_set_lbp_feature, test_set_vgg16_feature])

In [94]:
training_set_label_prediction_feature_fusion_positive = training_set_label_prediction_feature_fusion[training_set_label == 1]
training_set_label_prediction_feature_fusion_negative = training_set_label_prediction_feature_fusion[training_set_label == 0]

training_set_label_prediction_feature_fusion_positive_true = training_set_label_prediction_feature_fusion_positive[training_set_label_prediction_feature_fusion_positive >= 0.5]
training_set_label_prediction_feature_fusion_positive_false = training_set_label_prediction_feature_fusion_positive[training_set_label_prediction_feature_fusion_positive < 0.5]
training_set_label_prediction_feature_fusion_negative_true = training_set_label_prediction_feature_fusion_negative[training_set_label_prediction_feature_fusion_negative < 0.5]
training_set_label_prediction_feature_fusion_negative_false = training_set_label_prediction_feature_fusion_negative[training_set_label_prediction_feature_fusion_negative >= 0.5]

print(
    '''Feature Fusion+FC Training Set 混淆矩阵 Training Set
    real/predict   True    False
    
    smoke          {0}     {1}
    no smoke       {2}     {3}
    检测率：{4}
    虚警率：{5}'''.format(
    training_set_label_prediction_feature_fusion_positive_true.shape[0],
    training_set_label_prediction_feature_fusion_positive_false.shape[0],
    training_set_label_prediction_feature_fusion_negative_true.shape[0],
    training_set_label_prediction_feature_fusion_negative_false.shape[0],
    training_set_label_prediction_feature_fusion_positive_true.shape[0] / training_set_label_prediction_feature_fusion_positive.shape[0],
    training_set_label_prediction_feature_fusion_negative_false.shape[0] / training_set_label_prediction_feature_fusion_negative.shape[0]
    )
)

混淆矩阵 Training Set
    real/predict   True    False
    
    smoke          552     0
    no smoke       831     0
    检测率：1.0
    虚警率：0.0


In [95]:
test_set_label_prediction_feature_fusion_positive = test_set_label_prediction_feature_fusion[test_set_label == 1]
test_set_label_prediction_feature_fusion_negative = test_set_label_prediction_feature_fusion[test_set_label == 0]

test_set_label_prediction_feature_fusion_positive_true = test_set_label_prediction_feature_fusion_positive[test_set_label_prediction_feature_fusion_positive >= 0.5]
test_set_label_prediction_feature_fusion_positive_false = test_set_label_prediction_feature_fusion_positive[test_set_label_prediction_feature_fusion_positive < 0.5]
test_set_label_prediction_feature_fusion_negative_true = test_set_label_prediction_feature_fusion_negative[test_set_label_prediction_feature_fusion_negative < 0.5]
test_set_label_prediction_feature_fusion_negative_false = test_set_label_prediction_feature_fusion_negative[test_set_label_prediction_feature_fusion_negative >= 0.5]

print(
    '''Feature Fusion+FC Test Set 混淆矩阵 test Set
    real/predict   True    False
    
    smoke          {0}     {1}
    no smoke       {2}     {3}
    检测率：{4}
    虚警率：{5}'''.format(
    test_set_label_prediction_feature_fusion_positive_true.shape[0],
    test_set_label_prediction_feature_fusion_positive_false.shape[0],
    test_set_label_prediction_feature_fusion_negative_true.shape[0],
    test_set_label_prediction_feature_fusion_negative_false.shape[0],
    test_set_label_prediction_feature_fusion_positive_true.shape[0] / test_set_label_prediction_feature_fusion_positive.shape[0],
    test_set_label_prediction_feature_fusion_negative_false.shape[0] / test_set_label_prediction_feature_fusion_negative.shape[0]
    )
)

混淆矩阵 test Set
    real/predict   True    False
    
    smoke          684     4
    no smoke       785     32
    检测率：0.9941860465116279
    虚警率：0.03916768665850673
